In [1]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
  inflating: dataset/ODI(BABAR AZAM).csv  
  inflating: dataset/ODI(SHAHEEN SHAH AFRIDI).csv  
  inflating: dataset/ODIs_bat.csv    
  inflating: dataset/ODIs_bowl.csv   
  inflating: dataset/ODI_ALL.csv     
  inflating: dataset/T20I(BABAR AZAM).csv  
  inflating: dataset/T20i(SHAHEEN SHAH AFRIDI).csv  
  inflating: dataset/T20Is_ALL.csv   
  inflating: dataset/T20Is_bat.csv   
  inflating: dataset/T20Is_bowl.csv  
  inflating: dataset/TEST(BABAR AZAM).csv  
  inflating: dataset/TEST(SHAHEEN SHAH AFRIDI).csv  
  inflating: dataset/Tests_bat.csv   
  inflating: dataset/Tests_bowl.csv  
  inflating: dataset/TEST_ALL.csv    


In [2]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.decomposition import PCA
def Clustering(df,x):
    pca = PCA()
    components = pca.fit_transform(x)
    c1= components[:,0]
    c2= components[:,1]
    pcadata = pd.DataFrame(data={"0":c1,"1":c2})

    KM = KMeans(n_clusters = 2)
    Labels1 = KM.fit_predict(pcadata)
    print(Labels1)
    df["Label"] = Labels1
    return df

def remove_outlier(df_in, col):
    q1 = df_in[col].quantile(0.25)
    q3 = df_in[col].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col] > fence_low) & (df_in[col] < fence_high)]
    return df_out


Generate labels and preprocess

In [3]:
import pandas as pd

paths = ['dataset/Tests_bat.csv', 'dataset/ODIs_bat.csv', 'dataset/T20Is_bat.csv']

# Load datasets and assign a unique label to each
df = pd.concat([pd.read_csv(path).assign(format=format_name) for format_name, path in zip(['Test', 'ODI', 'T20I'], paths)], ignore_index=True)

df

,Player,Span,Mat,Inns,NO,Runs,HS,Ave,BF,SR,100,50,0,4s,6s,Unnamed: 15,format
0,Azhar Ali,2013-2022,73,136,7,5331,302*,41.32,12499,42.65,15,22,14,510,19,NaN,Test
1,Babar Azam,2016-2024,54,98,9,3962,196,44.51,7252,54.63,9,26,8,459,23,NaN,Test
2,Asad Shafiq,2013-2020,61,104,3,3766,137,37.28,7479,50.35,10,21,10,408,24,NaN,Test
3,Younis Khan,2013-2017,39,75,8,3534,218,52.74,6930,50.99,14,7,5,328,33,NaN,Test
4,Sarfaraz Ahmed,2013-2023,53,93,14,3025,118,38.29,4287,70.56,4,21,4,309,9,NaN,Test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,Asad Ali,2013-2013,2,-,-,-,-,-,-,-,-,-,-,-,-,NaN,T20I
235,Mubasir Khan,2023-2023,1,-,-,-,-,-,-,-,-,-,-,-,-,NaN,T20I
236,Muhammad Musa,2019-2020,2,-,-,-,-,-,-,-,-,-,-,-,-,NaN,T20I
237,Waqas Maqsood,2018-2018,1,-,-,-,-,-,-,-,-,-,-,-,-,NaN,T20I


In [4]:
df.dtypes

,0
Player,object
Span,object
Mat,int64
Inns,object
NO,object
Runs,object
HS,object
Ave,object
BF,object
SR,object


In [5]:
df.isnull().sum()

,0
Player,0
Span,0
Mat,0
Inns,0
NO,0
Runs,0
HS,0
Ave,0
BF,0
SR,0


In [6]:
# Rename columns
df.rename(columns = {'Mat':'Matches', 'Ave':'AVG'}, inplace = True)
# remove 'unnamed' columns
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
# remove rows with "-"
df = df.loc[df["AVG"] != "-"]

# Convert numeric columns (those that should be int or float)
numeric_columns = ['Matches', 'Inns', 'NO', 'Runs', 'AVG', 'BF', 'SR', '100', '50', '0', '4s', '6s']
# Convert to numeric, coercing errors and filling NaNs
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
# no nulls present
# convert to integers
df['Matches'] = df['Matches'].astype(int)
df['Inns'] = df['Inns'].astype(int)
df['NO'] = df['NO'].astype(int)
df['Runs'] = df['Runs'].astype(int)
df['BF'] = df['BF'].astype(int)

# convert to float
df['AVG'] = df['AVG'].astype(float)
df['SR'] = df['SR'].astype(float)

# convert to string
df['Player'] = df['Player'].astype(str)
df['Span'] = df['Span'].astype(str)
datasetBatsman = df[["Matches",'Inns', 'NO','AVG', 'SR', '50', '0', '4s', '6s']]
df = Clustering(df,datasetBatsman)
print(df.dtypes)
print(df)
df.to_csv('batting.csv', index=False)

[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Player      object
Span        object
Matches      int64
Inns         int64
NO           int64
Runs         int64
HS          object
AVG        float64
BF           int64
SR         float64
100          int64
50           int64
0            int64
4s           int64
6s           int64
format      object
Label        int32
dtype: object
               Player       Span  Matches  Inns  NO  Runs    HS    AVG     BF  \
0           Azhar Ali  2013-2022       73   136   7  5331  302*  41.32  12499   
1          Babar Azam  2016-2024       54    98   9  3962   196  44.51   7

<ipython-input-6-2279ac56c6cd>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')
<ipython-input-6-2279ac56c6cd>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Matches'] = df['Matches'].astype(int)
<ipython-input-6-2279ac56c6cd>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:

Generate labels and preprocess bowling


In [7]:
paths = ['dataset/Tests_bowl.csv', 'dataset/ODIs_bowl.csv', 'dataset/T20Is_bowl.csv']

# Load datasets and assign a format label to each
df1 = pd.concat([pd.read_csv(path).assign(format=format_name) for format_name, path in zip(['Test', 'ODI', 'T20I'], paths)], ignore_index=True)


df1


,Player,Span,Mat,Inns,Overs,Mdns,Runs,Wkts,BBI,BBM,Ave,Econ,SR,5,10,Unnamed: 15,format,4,Unnamed: 14
0,Yasir Shah,2014-2022,48,89,2375.5,359,7657,244,8/41,14/184,31.38,3.22,58.4,16,3,NaN,Test,NaN,NaN
1,Shaheen Shah Afridi,2018-2024,30,52,1001.0,205,3115,115,6/51,10/94,27.08,3.11,52.2,4,1,NaN,Test,NaN,NaN
2,Mohammad Abbas,2017-2021,25,44,855.4,261,2072,90,5/33,10/95,23.02,2.42,57.0,4,1,NaN,Test,NaN,NaN
3,Hasan Ali,2017-2024,24,46,715.5,164,2185,80,5/27,10/114,27.31,3.05,53.6,6,1,NaN,Test,NaN,NaN
4,Mohammad Amir,2016-2019,22,40,792.0,189,2143,68,6/44,7/64,31.51,2.70,69.8,1,0,NaN,Test,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,Sharjeel Khan,2013-2021,21,-,-,-,-,-,-,NaN,-,-,-,-,NaN,NaN,T20I,-,NaN
235,Tayyab Tahir,2023-2023,3,-,-,-,-,-,-,NaN,-,-,-,-,NaN,NaN,T20I,-,NaN
236,Umar Akmal,2012-2019,44,-,-,-,-,-,-,NaN,-,-,-,-,NaN,NaN,T20I,-,NaN
237,Umar Amin,2013-2018,14,-,-,-,-,-,-,NaN,-,-,-,-,NaN,NaN,T20I,-,NaN


In [8]:
df1.dtypes

,0
Player,object
Span,object
Mat,int64
Inns,object
Overs,object
Mdns,object
Runs,object
Wkts,object
BBI,object
BBM,object


In [9]:
# Rename columns as needed
df1.rename(columns={'Mat': 'Matches', 'Econ': 'Eco', 'Ave': 'AVG', 'SR': 'sr'}, inplace=True)

# Remove 'Unnamed' columns
df1.drop(df1.columns[df1.columns.str.contains('unnamed', case=False)], axis=1, inplace=True)

# Remove rows with "-" in 'AVG' column
df1 = df1.loc[df1['AVG'] != "-"]

# Convert numeric columns (those that should be int or float)
numeric_columns = ['Matches', 'Inns', 'Overs', 'Mdns', 'Runs', 'Wkts', 'AVG', 'Eco', 'sr', '5', '4', '10']

# Convert to numeric, coercing errors and filling NaNs
df1[numeric_columns] = df1[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Fill NaNs for columns that can reasonably have zero as a default
df1[['4', '10', '5']] = df1[['4', '10', '5']].fillna(0).astype('int64')

# Fill NaNs for other columns if necessary, here for example with 'BBM' and 'BBI'
df1['BBM'] = df1['BBM'].fillna('nan')
df1['BBI'] = df1['BBI'].fillna('nan')

# Convert the appropriate columns to integer and float types
df1['Matches'] = df1['Matches'].astype('int64')
df1['Inns'] = df1['Inns'].astype('int64')
df1['Runs'] = df1['Runs'].astype('int64')
df1['Wkts'] = df1['Wkts'].astype('int64')

# Convert columns like 'AVG', 'Eco', 'sr', and 'Overs' to float
df1['AVG'] = df1['AVG'].astype('float64')
df1['Eco'] = df1['Eco'].astype('float64')
df1['sr'] = df1['sr'].astype('float64')
df1['Overs'] = df1['Overs'].astype('float64')

# Convert remaining columns to strings as needed
df1['Player'] = df1['Player'].astype('str')
df1['Span'] = df1['Span'].astype('str')
df1['BBM'] = df1['BBM'].astype('str')
df1['BBI'] = df1['BBI'].astype('str')

# move the 4 column to end
df1 = df1[['Player', 'Span', 'Matches', 'Inns', 'Overs', 'Mdns', 'Runs', 'Wkts', 'BBI', 'BBM', 'AVG', 'Eco', 'sr', '5', '10', '4']]

datasetBowling = df1[["Matches", 'Inns', 'Overs', 'Runs', 'Wkts', 'AVG', 'Eco', 'sr']]
df1 = Clustering(df1, datasetBowling)

# Print the data types and the dataframe
print(df1.dtypes)
print(df1)

# Save to CSV
df1.to_csv('bowling.csv', index=False)


[0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
Player      object
Span        object
Matches      int64
Inns         int64
Overs      float64
Mdns         int64
Runs         int64
Wkts         int64
BBI         object
BBM         object
AVG        float64
Eco        float64
sr         float64
5            int64
10           int64
4            int64
Label        int32
dtype: object
                  Player       Span  Matches  Inns   Overs  Mdns  Runs  Wkts  \
0             Yasir Shah  2014-2022       48    89  2375.5   359  7657   244   
1    Shaheen Shah Afridi  2018-2024       30    52  1001.0   205  3115   115   
2         Mohammad Abbas  2017-2021       25    44   855.4   261  2072    90   
3              Hasan Ali  2017-2024       

<ipython-input-9-3d26ae31baf8>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[numeric_columns] = df1[numeric_columns].apply(pd.to_numeric, errors='coerce')
<ipython-input-9-3d26ae31baf8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[['4', '10', '5']] = df1[['4', '10', '5']].fillna(0).astype('int64')
<ipython-input-9-3d26ae31baf8>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

Generate labels and preprocess Allrounder

In [10]:
paths = ['dataset/TEST_ALL.csv', 'dataset/ODI_ALL.csv', 'dataset/T20Is_ALL.csv']

# Load datasets and assign a unique label to each
df2 = pd.concat([pd.read_csv(path).assign(format=format_name) for format_name, path in zip(['Test', 'ODI', 'T20I'], paths)], ignore_index=True)

df2

,Player,Span,Mat,Runs,HS,Bat Av,100,Wkts,BBI,Bowl Av,5,Ct,St,Ave Diff,Unnamed: 14,format
0,Azhar Ali,2013-2022,73,5331,302*,41.32,15,7,2/35,81.00,0,51,0,-39.67,NaN,Test
1,Asad Shafiq,2013-2020,61,3766,137,37.28,10,3,1/7,65.33,0,64,0,-28.04,NaN,Test
2,Babar Azam,2016-2024,54,3962,196,44.51,9,2,1/1,21.00,0,41,0,23.51,NaN,Test
3,Sarfaraz Ahmed,2013-2023,53,3025,118,38.29,4,-,-,-,-,156,22,-,NaN,Test
4,Yasir Shah,2014-2022,48,918,113,14.12,1,244,8/41,31.38,16,24,0,-17.25,NaN,Test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
234,Haseebullah Khan,2024-2024,1,0,0,0.00,0,-,-,-,-,1,0,-,NaN,T20I
235,Mubasir Khan,2023-2023,1,-,-,-,-,-,-,-,-,0,0,-,NaN,T20I
236,Nauman Anwar,2015-2015,1,18,18,18.00,0,-,-,-,-,0,0,-,NaN,T20I
237,Waqas Maqsood,2018-2018,1,-,-,-,-,2,2/21,10.50,0,0,0,-,NaN,T20I


In [11]:
df2.dtypes

,0
Player,object
Span,object
Mat,int64
Runs,object
HS,object
Bat Av,object
100,object
Wkts,object
BBI,object
Bowl Av,object


In [12]:
df2.isnull().sum()

,0
Player,0
Span,0
Mat,0
Runs,0
HS,0
Bat Av,0
100,0
Wkts,0
BBI,0
Bowl Av,0


In [13]:
# Rename columns
df2.rename(columns = {'Mat':'Matches', 'Bat Av':'Bat AVG', 'Bowl Av':'Bowl AVG', 'Wkts':'wcts'}, inplace = True)
# Remove 'Unnamed' columns
df2.drop(df2.columns[df2.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
# remove rows with "-"
df2 = df2.loc[df2["wcts"] != "-"]
df2 = df2.loc[df2["Bowl AVG"] != "-"]
df2 = df2.loc[df2["Bat AVG"] != "-"]
# numeric columns
numeric_columns = ['Matches', 'Runs', 'Bat AVG', 'Bowl AVG', 'wcts', 'Ct', 'St']
df2[numeric_columns] = df2[numeric_columns].apply(pd.to_numeric, errors='coerce')
# no nulls present
# convert to integers
df2['Matches'] = df2['Matches'].astype(int)
df2['Runs'] = df2['Runs'].astype(int)

# convert to float
df2['Bat AVG'] = df2['Bat AVG'].astype(float)
df2['Bowl AVG'] = df2['Bowl AVG'].astype(float)
df2['Ave Diff'] = df2['Ave Diff'].astype(float)

# convert to strings
df2['Player'] = df2['Player'].astype(str)
df2['Span'] = df2['Span'].astype(str)
df2['HS'] = df2['HS'].astype(str)
df2['BBI'] = df2['BBI'].astype(str)
# remove outlier
df2=remove_outlier(df2,"Runs")
datasetAllrounder = df2[['Matches','Runs','Bat AVG','Bowl AVG','wcts']]
df2 = Clustering(df2,datasetAllrounder)
print(df2.dtypes)
print(df2)
df2.to_csv('allrounder.csv', index=False)

[0 1 1 0 0 1 1 1 0 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 0 0 0 1 1 1 1 1 1 0 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 0 1 1 0 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]
Player       object
Span         object
Matches       int64
Runs          int64
HS           object
Bat AVG     float64
100          object
wcts          int64
BBI          object
Bowl AVG    float64
5            object
Ct            int64
St            int64
Ave Diff    float64
format       object
Label         int32
dtype: object
                  Player       Span  Matches  Runs   HS  Bat AVG 100  wcts  \
4             Yasir Shah  2014-2022       48   918  113    14.12   1   244   
9    Shaheen Shah Afridi  2018-2024       30   213  29*     7.34   0   115   
11        Mohammad Abbas  2017-2021       25   110   29     5.50   0    90   
12             Hasan Ali  2017-2024       24   382   30    11.93   0    80   
14         Mohammad Amir  2016-2019       22   

<ipython-input-2-1fa3310a07b9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Label"] = Labels1


Batch process rest of the player profile csvs

In [14]:
paths = ['dataset/ODI(BABAR AZAM).csv', 'dataset/ODI(SHAHEEN SHAH AFRIDI).csv', 'dataset/T20I(BABAR AZAM).csv', 'dataset/T20i(SHAHEEN SHAH AFRIDI).csv', 'dataset/TEST(BABAR AZAM).csv', 'dataset/TEST(SHAHEEN SHAH AFRIDI).csv']

for dataset in paths:
    # load that csv
    data = pd.read_csv(dataset)
    # remove 'unnamed' columns
    data.drop(data.columns[data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
    # drop when runs are '-'
    data = data.loc[data["Runs"] != "-"]
    # save the dataset to root with the same name
    data.to_csv(dataset.split('/')[1], index=False)

In [15]:
!zip -r preprocessed.zip *.csv

  adding: allrounder.csv (deflated 59%)
  adding: batting.csv (deflated 61%)
  adding: bowling.csv (deflated 60%)
  adding: ODI(BABAR AZAM).csv (deflated 64%)
  adding: ODI(SHAHEEN SHAH AFRIDI).csv (deflated 62%)
  adding: T20I(BABAR AZAM).csv (deflated 66%)
  adding: T20i(SHAHEEN SHAH AFRIDI).csv (deflated 68%)
  adding: TEST(BABAR AZAM).csv (deflated 57%)
  adding: TEST(SHAHEEN SHAH AFRIDI).csv (deflated 52%)
